In [1]:
import torch
from transformers import pipeline
import sys
import os
from scipy.io import wavfile
from bs4 import BeautifulSoup

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
transcribe = pipeline(task="automatic-speech-recognition", model="BUT-FIT/whisper-ATC-czech-full", chunk_length_s=30, device=device)
# transcribe.model.config.forced_decoder_ids = transcribe.tokenizer.get_decoder_prompt_ids(task="transcribe", language="czech")
# setting the parameters of model for correct working
transcribe.generation_config.language = "czech"
transcribe.generation_config.task = "transcribe"
transcribe.generation_config.forced_decoder_ids = None


In [3]:
audio_path = "/mnt/datadisk/ATCO2-ASRdataset-v1_final/DATA_nonEN/"
file="LKTB_BRNO_Tower_119_605MHz_20201028_140027.wav"
print('Transcription:', transcribe(os.path.join(audio_path,file)))

f = open(os.path.join(audio_path,file).replace('.wav','.xml'), "r")
xml_data = f.read()
soup = BeautifulSoup(xml_data, "xml")
print(soup.find("text").text)

/home/johnny/.local/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, 50283], [2, 50359], [3, 50363]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Transcription: {'text': 'Oscar Kilo Charlie Echo Sierra informace November QNH jedna nula jedna šest a jste povoleni za VFR na Sierru squawk čtyři tři dva nula\n'}
Oscar Kilo Charlie Echo Sierra informace november QNH jedna nula jedna šest a jste povolení za VFR na sierru squawk čtyři tři dva nula


In [4]:
def getTranscription(file_path, display_audio=False, print_ts=False) -> tuple:
    # model transcription
    model_ts = transcribe(file_path)['text'].rstrip()
    
    if (os.path.exists(file_path.replace('.wav','.xml'))):
        f = open(file_path.replace('.wav','.xml'), "r")
        xml_data = f.read()
        soup = BeautifulSoup(xml_data, "xml")
        xml_ts = soup.find("text").text
    
    if (display_audio):
        display(Audio(filename=file_path))
    
    if (print_ts):
        print('Model Transcription:', model_ts)
        print('XML Transcription:', xml_ts)
        
    return model_ts, xml_ts

In [5]:
import json, re, glob

def loadJsonFile(file_path):
    if (os.path.exists(file_path)):
        with open(file_path) as json_file:
            return json.load(json_file)
    else: 
        return {"ProcessedFiles":[],"Records":[]}

def saveJsonFile(file_path, data):  
    with open(file_path, 'w') as outfile:
        json.dump(data, outfile, indent=None, separators=(",", ": "))

jsonpath = "./data.json"
data = loadJsonFile(jsonpath)
transcribed_files = data['ProcessedFiles']

for path in glob.glob("/mnt/datadisk/ATCO2-ASRdataset-v1_final/DATA_nonEN/*.wav"):
    filename = os.path.basename(path)
    if (filename not in transcribed_files):
        # so far, process only cs recordings (meaning BRNO and RUZYNE in this case)
        r = re.search(r'^[^_]+_([^_]+)_([^_]+)', filename)
        
        if (r.group(1) in ['RUZYNE', 'BRNO']):
            model_ts, xml_ts = getTranscription(path)
            # read the wav file
            sample_rate, wavdata = wavfile.read(path)
            
            data['ProcessedFiles'].append(filename)
            data['Records'].append({"Audio_path": path, "Name":filename, "Array": wavdata.tolist(), "Sample_rate": sample_rate, "Model_ts": transcribe(path)['text'].rstrip(), "XML_ts" :xml_ts})
            
            print('Processed:', filename)

saveJsonFile('data.json', data)

/home/johnny/.local/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/home/johnny/.local/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Processed: LKTB_BRNO_Tower_119_605MHz_20201028_153007.wav


/home/johnny/.local/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/home/johnny/.local/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Processed: LKPR_RUZYNE_Tower_134_560MHz_20201029_151000.wav


/home/johnny/.local/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/home/johnny/.local/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


...

Processed: LKPR_RUZYNE_Tower_134_560MHz_20201026_155816.wav


/home/johnny/.local/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/home/johnny/.local/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Processed: LKPR_RUZYNE_Radar_120_520MHz_20201025_125020.wav


/home/johnny/.local/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/home/johnny/.local/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Processed: LKPR_RUZYNE_Tower_134_560MHz_20201025_132420.wav


In [28]:
from ipywidgets import widgets
from IPython.display import display

jsonpath = "./data.json"
jsondata = loadJsonFile(jsonpath)

for item in jsondata['Records']:
    # Create a text input widget
    text_correct_long = widgets.Text(
        placeholder="Long correct text",
        description="Model transcript:",
        value=item['Model_ts'],
        layout=widgets.Layout(width='1500px'),
        disabled=True
    )

    text_correct_short = widgets.Text(
        placeholder="Short correct text",
        description="XML transcript:",
        value=item['XML_ts'],
        layout=widgets.Layout(width='1500px'),
        disabled=True
    )
    
    text_new_ts = widgets.Text(
        placeholder="New transcript",
        disabled=False,
        value=item['XML_ts'],
        layout=widgets.Layout(width='1500px'),
        description="New transcript:"
    )
    # Create a button widget
    submit_button = widgets.Button(description="Submit")

    def save_new_jsonval():
        item['XML_ts'] = "thisismyspecialtext"
        # print('New transcript:', item['mytranscript'])
        
    # Set the button's click event to the function
    submit_button.on_click(lambda _: save_new_jsonval)

    # Display the widgets
    display(text_correct_long,text_correct_short, text_new_ts,submit_button)

# Create a button widget
fin_btn = widgets.Button(description="Write to JSON")
    
# Set the button's click event to the function
fin_btn.on_click(lambda _: saveJsonFile(jsonpath,jsondata))
display(fin_btn)

Text(value='Oscar Whiskey India Tři Sedm QNH jedna nula jedna šest maximumě dva tisíce pět set stop povolení T…

Text(value='Oscar Whiskey India Tři Sedm QNH jedna nula jedna šest maximálně dva tisíce pět set stop povolení …

Text(value='Oscar Whiskey India Tři Sedm QNH jedna nula jedna šest maximálně dva tisíce pět set stop povolení …

Button(description='Submit', style=ButtonStyle())

Text(value='Kunovice Nula Jedna v CTR uzlíní maximum dva tisíce pět set stop QNH jedna nula jedna osm', descri…

...

Text(value='po Alphě pokračuju pravý baseleg dráhy dva čtyři a ohlásím finále Oscar Kilo Papa Alpha Zulu', des…

Text(value='po alfě pokračuju pravý base lag dráhy dva čtyři a ohlásím [hes] finále Oskar Kilo Papa Alfa Zulu'…

Text(value='po alfě pokračuju pravý base lag dráhy dva čtyři a ohlásím [hes] finále Oskar Kilo Papa Alfa Zulu'…

Button(description='Submit', style=ButtonStyle())

Button(description='Write to JSON', style=ButtonStyle())

In [12]:
dd = {
    "name": "Jmeno",
    "surname": "Prijmeni", 
    'dalsidata': [
        {"name": "Jmeno1", "surname": "Prijmeni1"},
        {"name": "Jmeno2", "surname": "Prijmeni2"},
    ]
}
with open('test.json', 'w') as outfile:
    json.dump(dd,outfile)


In [15]:
ddload = json.load(open('test.json'))
print(ddload)
for item in ddload['dalsidata']:
    print(item['name'], item['surname'])
    item['name'] = f'New{item["name"]}'
    item['surname'] = f'New{item["surname"]}'
print(ddload)

{'name': 'Jmeno', 'surname': 'Prijmeni', 'dalsidata': [{'name': 'Jmeno1', 'surname': 'Prijmeni1'}, {'name': 'Jmeno2', 'surname': 'Prijmeni2'}]}
Jmeno1 Prijmeni1
Jmeno2 Prijmeni2
{'name': 'Jmeno', 'surname': 'Prijmeni', 'dalsidata': [{'name': 'NewJmeno1', 'surname': 'NewPrijmeni1'}, {'name': 'NewJmeno2', 'surname': 'NewPrijmeni2'}]}


In [10]:
ddload['name'] = 'Pavel'
with open('test.json', 'w') as outfile:
    json.dump(ddload,outfile)